# Battle of the Neighborhoods

### Coursera Captone Project

## Introduction

With over 150 million daily coffee drinkers, and as someone who will be moving to New Your City and is an avid coffee drinker it is very important to move to an area with a lot of coffe shops nearby. Varity is key. If there are to few shops then they will always be too busy and someone whould have to wait a long time to get their coffee.  One does not want to wait, but wants to be able to get in and out so they can go on with their day.   In this project I will be comparing different areas of New York to determine which area has the best options avaialbe for a varity of coffee shops. The more coffee shops in the area the better.

## Data

In order to answer the above questions, data on New York City neighborhoods, boroughs to include boundaries, latitude, longitude, restaurants, and restaurant ratings and tips are required.

New York City data containing the neighborhoods and boroughs, latitudes, and longitudes will be obtained from the data source: https://cocl.us/new_york_dataset

All data related to locations and quality of  coffee shops will be obtained via the FourSquare API utilized via the Request library in Python

## Methodology:

1.  Import all important data packages that will be needed
2.  Import all data
3.  Map and Analyze New York Neighborhoods
4.  Utilize Foursquare API to explore Neighborhoods
5.  Discuss Results
6.  Observations and Recommendations 
7.  Conclusion 

## 1.  Import Data Packages

In [29]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium==0.5.0
import folium # plotting library

import urllib.request as request

print('Libraries imported.')

Libraries imported.


## 2. Import Data Sets and Transform to pandas Dataframe

In [36]:
with request.urlopen('https://cocl.us/new_york_dataset') as response:
    source = response.read()
    ny_data=json.loads(source)

In [37]:
print(ny_data)

{'type': 'FeatureCollection', 'totalFeatures': 306, 'features': [{'type': 'Feature', 'id': 'nyu_2451_34572.1', 'geometry': {'type': 'Point', 'coordinates': [-73.84720052054902, 40.89470517661]}, 'geometry_name': 'geom', 'properties': {'name': 'Wakefield', 'stacked': 1, 'annoline1': 'Wakefield', 'annoline2': None, 'annoline3': None, 'annoangle': 0.0, 'borough': 'Bronx', 'bbox': [-73.84720052054902, 40.89470517661, -73.84720052054902, 40.89470517661]}}, {'type': 'Feature', 'id': 'nyu_2451_34572.2', 'geometry': {'type': 'Point', 'coordinates': [-73.82993910812398, 40.87429419303012]}, 'geometry_name': 'geom', 'properties': {'name': 'Co-op City', 'stacked': 2, 'annoline1': 'Co-op', 'annoline2': 'City', 'annoline3': None, 'annoangle': 0.0, 'borough': 'Bronx', 'bbox': [-73.82993910812398, 40.87429419303012, -73.82993910812398, 40.87429419303012]}}, {'type': 'Feature', 'id': 'nyu_2451_34572.3', 'geometry': {'type': 'Point', 'coordinates': [-73.82780644716412, 40.887555677350775]}, 'geometry_n

In [42]:
neighborhoods_data = ny_data['features']

In [43]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [44]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [45]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [46]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [59]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [60]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


## 3.  Map and Analize New York Data from Dataframe

In [61]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [62]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## 4.  Utilize Foursquare to explore the Neighborhoods.

In [84]:

CLIENT_ID = 'MV2OBZBLUVI5FNK04SDIIAZUNFENZVBHD41LKTSMFD3HTCNT' # your Foursquare ID
CLIENT_SECRET = '3DKW4XEKGG3XEFN1SDJA0GHRTE5GQDYKWBZDH5MOIELPSBQC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MV2OBZBLUVI5FNK04SDIIAZUNFENZVBHD41LKTSMFD3HTCNT
CLIENT_SECRET:3DKW4XEKGG3XEFN1SDJA0GHRTE5GQDYKWBZDH5MOIELPSBQC


In [89]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)


In [90]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc42a26ea7b29205e43783a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 114,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57f0689d498e7d49d9189369',
       'name': 'The Bar Room at Temple Court',
       'location': {'address': '123 Nassau St',
        'lat': 40.7114477287544,
        'lng': -74.00680157032005,
        'labe

In [91]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [94]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,The Bar Room at Temple Court,Hotel Bar,40.711448,-74.006802
1,"The Beekman, A Thompson Hotel",Hotel,40.711173,-74.006702
2,Alba Dry Cleaner & Tailor,Laundry Service,40.711434,-74.006272
3,City Hall Park,Park,40.711893,-74.007792
4,The Wooly Daily,Coffee Shop,40.712137,-74.008395


In [106]:
nearby_venues.shape

(100, 4)

In [153]:
nearby_venues.groupby(['categories']).size()

categories
American Restaurant                3
Antique Shop                       1
Asian Restaurant                   1
Baby Store                         1
Bagel Shop                         1
Bakery                             1
Bookstore                          1
Boxing Gym                         1
Breakfast Spot                     1
Bubble Tea Shop                    2
Building                           1
Burger Joint                       2
Burrito Place                      1
Café                               3
Cajun / Creole Restaurant          1
Clothing Store                     1
Cocktail Bar                       3
Coffee Shop                        4
Comic Shop                         1
Cosmetics Shop                     1
Coworking Space                    1
Cuban Restaurant                   2
Dance Studio                       1
Deli / Bodega                      1
Discount Store                     1
Electronics Store                  1
Falafel Restaurant         

##### Map Nearby Venues

In [166]:
locations = nearby_venues[['lat','lng']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[40.713652845301894, -74.0088038953017]

In [186]:
map_venues = folium.Map(location=[40.71, -74.00], zoom_start=16)

nearby_venues.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lng"]], 
                                              radius=8, color= "Blue")
                                             .add_to(map_venues), axis=1)


map_venues

## 5.  Reults

The results are in... There are many coffee shops and other awesome things to do in New York.  If someone is really wanting to move to New York there are plenty of places that would be aweosme to move.

## 6. Observations and Recommedation

It has been observed that there are great coffee shops all over New York.  My recommendation..... Do not move to New York.  No amount of coffee shops or anything else for that matter are worth moving someplace where there are just way to many people!

## 7.  Conclusion

In conclusion this has been a fantastic class and this capstone project was a blast.  But still.... you won't find me analysing data to move to New York.  I'm happy were I'm at and the mountains are the best place to be.